# 3. FINAL DATA PROFILING - CLEANED DATASET

This notebook performs data profiling on the **fully cleaned dataset** after running:
- **1. Data Profiling & Quality Assessment** - Initial inspection
- **2a. Data Transformation & Standardization** - Text normalization, typo correction, wrangling
- **2b. Error Detection & Correction** - Address repair, missing values imputation, outlier detection
- **2c. Data Deduplication** - Exact and near-duplicate removal

## Contents:
1. Load the cleaned dataset
2. Data overview and structure
3. Completeness analysis (missing values)
4. Statistical profiling (numerical columns)
5. Categorical profiling (value distributions)
6. Automatic profiling with ydata_profiling
7. Before vs After comparison

## 3.1 Imports and Setup

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import json

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 140)
%matplotlib inline

## 3.2 Load the Cleaned Dataset

In [ ]:
# Load the final deduplicated dataset (after all cleaning steps)
MILANO_CLEAN = pd.read_csv("MILANO_deduplicated.csv", sep=";")
print(f"Cleaned dataset shape: {MILANO_CLEAN.shape[0]} rows x {MILANO_CLEAN.shape[1]} columns")

# Also load original for comparison
MILANO_ORIG = pd.read_csv("Comune-di-Milano-Pubblici-esercizi(in)-2.csv", sep=";")
print(f"Original dataset shape: {MILANO_ORIG.shape[0]} rows x {MILANO_ORIG.shape[1]} columns")

print(f"\nRows removed during cleaning: {MILANO_ORIG.shape[0] - MILANO_CLEAN.shape[0]}")
print(f"Columns added during cleaning: {MILANO_CLEAN.shape[1] - MILANO_ORIG.shape[1]}")

In [ ]:
# Preview cleaned data
MILANO_CLEAN.head()

## 3.3 Data Structure Overview

In [ ]:
# Column names
print("Columns in cleaned dataset:")
print(MILANO_CLEAN.columns.tolist())

In [ ]:
# Data types
print("Data types:")
MILANO_CLEAN.dtypes

In [ ]:
# Separate numerical and categorical columns
NUM = list(MILANO_CLEAN.select_dtypes(include=['int64', 'float64']).columns)
CAT = list(MILANO_CLEAN.select_dtypes(include=['bool', 'object']).columns)

print(f"Numerical columns ({len(NUM)}): {NUM}")
print(f"Categorical columns ({len(CAT)}): {CAT}")

---
# 4. COMPLETENESS ANALYSIS

Check for any remaining missing values after cleaning.

In [ ]:
# Missing values summary
null_counts = MILANO_CLEAN.isnull().sum()
null_pct = (null_counts / len(MILANO_CLEAN) * 100).round(2)

missing_df = pd.DataFrame({
    'Column': null_counts.index,
    'Missing Count': null_counts.values,
    'Missing %': null_pct.values
}).sort_values('Missing %', ascending=False)

print("=== MISSING VALUES IN CLEANED DATASET ===")
missing_with_nulls = missing_df[missing_df['Missing Count'] > 0]
if len(missing_with_nulls) > 0:
    display(missing_with_nulls)
else:
    print("✓ No missing values! Dataset is complete.")

In [ ]:
# Compare completeness: Before vs After
orig_missing = MILANO_ORIG.isnull().sum().sum()
clean_missing = MILANO_CLEAN.isnull().sum().sum()

orig_cells = MILANO_ORIG.shape[0] * MILANO_ORIG.shape[1]
clean_cells = MILANO_CLEAN.shape[0] * MILANO_CLEAN.shape[1]

print("=== COMPLETENESS COMPARISON ===")
print(f"Original dataset:")
print(f"  Total cells: {orig_cells}")
print(f"  Missing cells: {orig_missing} ({orig_missing/orig_cells*100:.2f}%)")
print(f"\nCleaned dataset:")
print(f"  Total cells: {clean_cells}")
print(f"  Missing cells: {clean_missing} ({clean_missing/clean_cells*100:.2f}%)")

---
# 5. NUMERICAL PROFILING

In [ ]:
# Descriptive statistics for numerical columns
MILANO_CLEAN[NUM].describe()

In [ ]:
# Profiling metrics for each numerical column
ROWS = len(MILANO_CLEAN)

num_profile = []
for col in NUM:
    count = MILANO_CLEAN[col].count()
    distinct = MILANO_CLEAN[col].nunique()
    uniqueness = distinct / ROWS
    distinctness = distinct / count if count > 0 else 0
    
    num_profile.append({
        'Column': col,
        'Count': count,
        'Missing': ROWS - count,
        'Distinct': distinct,
        'Uniqueness': round(uniqueness, 4),
        'Distinctness': round(distinctness, 4),
        'Min': MILANO_CLEAN[col].min(),
        'Max': MILANO_CLEAN[col].max(),
        'Mean': round(MILANO_CLEAN[col].mean(), 2),
        'Std': round(MILANO_CLEAN[col].std(), 2)
    })

pd.DataFrame(num_profile)

In [ ]:
# Distribution plots for numerical columns
for col in NUM:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Histogram
    MILANO_CLEAN[col].hist(bins=50, ax=axes[0])
    axes[0].set_title(f'Distribution of {col}')
    axes[0].set_xlabel(col)
    axes[0].set_ylabel('Frequency')
    
    # Boxplot
    axes[1].boxplot(MILANO_CLEAN[col].dropna())
    axes[1].set_title(f'Boxplot of {col}')
    axes[1].set_ylabel(col)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Correlation heatmap (if multiple numerical columns)
if len(NUM) > 1:
    plt.figure(figsize=(10, 8))
    sns.heatmap(MILANO_CLEAN[NUM].corr(), annot=True, cmap='coolwarm', center=0)
    plt.title("Correlation Heatmap - Numerical Columns")
    plt.tight_layout()
    plt.show()

---
# 6. CATEGORICAL PROFILING

In [ ]:
# Profiling metrics for categorical columns
cat_profile = []
for col in CAT:
    count = MILANO_CLEAN[col].count()
    distinct = MILANO_CLEAN[col].nunique()
    uniqueness = distinct / ROWS
    distinctness = distinct / count if count > 0 else 0
    mode_val = MILANO_CLEAN[col].mode()[0] if len(MILANO_CLEAN[col].mode()) > 0 else 'N/A'
    mode_freq = MILANO_CLEAN[col].value_counts().iloc[0] if count > 0 else 0
    constancy = mode_freq / count if count > 0 else 0
    
    cat_profile.append({
        'Column': col,
        'Count': count,
        'Missing': ROWS - count,
        'Distinct': distinct,
        'Uniqueness': round(uniqueness, 4),
        'Distinctness': round(distinctness, 4),
        'Mode': mode_val,
        'Mode Frequency': mode_freq,
        'Constancy': round(constancy, 4)
    })

pd.DataFrame(cat_profile)

In [ ]:
# Value counts for key categorical columns
key_cats = [
    "Tipo_macro", 
    "Forma commercio", 
    "Forma vendita", 
    "Tipo via",
    "ZD"
]

for c in key_cats:
    if c in MILANO_CLEAN.columns:
        print(f"\n=== {c} ===")
        display(MILANO_CLEAN[c].value_counts().head(10))

In [ ]:
# Distribution of Tipo_macro (if exists)
if 'Tipo_macro' in MILANO_CLEAN.columns:
    plt.figure(figsize=(10, 6))
    MILANO_CLEAN['Tipo_macro'].value_counts().plot(kind='bar')
    plt.title('Distribution of Tipo_macro (Macro-Categories)')
    plt.xlabel('Tipo_macro')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# Distribution by Zone (ZD)
if 'ZD' in MILANO_CLEAN.columns:
    plt.figure(figsize=(12, 5))
    MILANO_CLEAN['ZD'].value_counts().sort_index().plot(kind='bar')
    plt.title('Distribution of Establishments by Zone (ZD)')
    plt.xlabel('Zone (ZD)')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()

---
# 7. AUTOMATIC PROFILING WITH YDATA

In [ ]:
from ydata_profiling import ProfileReport

# Generate profiling report for cleaned dataset
PROFILE_CLEAN = ProfileReport(
    MILANO_CLEAN, 
    title="Profiling Report - Milan Public Establishments (CLEANED)",
    explorative=True
)
PROFILE_CLEAN

In [ ]:
# Save reports
PROFILE_CLEAN.to_file("MILANO_CLEANED_REPORT.html")
PROFILE_CLEAN.to_file("MILANO_CLEANED_REPORT.json")
print("Reports saved: MILANO_CLEANED_REPORT.html, MILANO_CLEANED_REPORT.json")

---
# 8. BEFORE VS AFTER COMPARISON

In [ ]:
# Summary comparison
comparison = pd.DataFrame({
    'Metric': [
        'Total Rows',
        'Total Columns',
        'Total Cells',
        'Missing Cells',
        'Missing %',
        'Completeness %'
    ],
    'Original': [
        MILANO_ORIG.shape[0],
        MILANO_ORIG.shape[1],
        MILANO_ORIG.shape[0] * MILANO_ORIG.shape[1],
        MILANO_ORIG.isnull().sum().sum(),
        f"{MILANO_ORIG.isnull().sum().sum() / (MILANO_ORIG.shape[0] * MILANO_ORIG.shape[1]) * 100:.2f}%",
        f"{100 - MILANO_ORIG.isnull().sum().sum() / (MILANO_ORIG.shape[0] * MILANO_ORIG.shape[1]) * 100:.2f}%"
    ],
    'Cleaned': [
        MILANO_CLEAN.shape[0],
        MILANO_CLEAN.shape[1],
        MILANO_CLEAN.shape[0] * MILANO_CLEAN.shape[1],
        MILANO_CLEAN.isnull().sum().sum(),
        f"{MILANO_CLEAN.isnull().sum().sum() / (MILANO_CLEAN.shape[0] * MILANO_CLEAN.shape[1]) * 100:.2f}%",
        f"{100 - MILANO_CLEAN.isnull().sum().sum() / (MILANO_CLEAN.shape[0] * MILANO_CLEAN.shape[1]) * 100:.2f}%"
    ]
})

print("=== BEFORE VS AFTER COMPARISON ===")
display(comparison)

In [ ]:
# Column-level missing values comparison
orig_missing_by_col = MILANO_ORIG.isnull().sum()
clean_missing_by_col = MILANO_CLEAN.isnull().sum()

# Get common columns
common_cols = [c for c in MILANO_ORIG.columns if c in MILANO_CLEAN.columns]

comparison_cols = pd.DataFrame({
    'Column': common_cols,
    'Original Missing': [orig_missing_by_col.get(c, 0) for c in common_cols],
    'Cleaned Missing': [clean_missing_by_col.get(c, 0) for c in common_cols]
})
comparison_cols['Improvement'] = comparison_cols['Original Missing'] - comparison_cols['Cleaned Missing']
comparison_cols = comparison_cols.sort_values('Improvement', ascending=False)

print("=== MISSING VALUES IMPROVEMENT BY COLUMN ===")
display(comparison_cols[comparison_cols['Improvement'] > 0])

In [ ]:
# Visual comparison of missing values
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Original
orig_pct = (MILANO_ORIG.isnull().sum() / len(MILANO_ORIG) * 100).sort_values(ascending=True)
orig_pct[orig_pct > 0].plot(kind='barh', ax=axes[0], color='salmon')
axes[0].set_title('Missing Values % - ORIGINAL')
axes[0].set_xlabel('Missing %')

# Cleaned
clean_pct = (MILANO_CLEAN.isnull().sum() / len(MILANO_CLEAN) * 100).sort_values(ascending=True)
if clean_pct[clean_pct > 0].empty:
    axes[1].text(0.5, 0.5, 'No Missing Values!\n✓ 100% Complete', 
                 ha='center', va='center', fontsize=16, color='green',
                 transform=axes[1].transAxes)
    axes[1].set_xlim(0, 1)
else:
    clean_pct[clean_pct > 0].plot(kind='barh', ax=axes[1], color='lightgreen')
axes[1].set_title('Missing Values % - CLEANED')
axes[1].set_xlabel('Missing %')

plt.tight_layout()
plt.show()

---
# 9. DATA QUALITY SUMMARY

In [ ]:
print("=" * 60)
print("FINAL DATA QUALITY SUMMARY")
print("=" * 60)

print(f"\n📊 DATASET SIZE:")
print(f"   Rows: {MILANO_CLEAN.shape[0]:,}")
print(f"   Columns: {MILANO_CLEAN.shape[1]}")
print(f"   Total cells: {MILANO_CLEAN.shape[0] * MILANO_CLEAN.shape[1]:,}")

completeness = 100 - (MILANO_CLEAN.isnull().sum().sum() / (MILANO_CLEAN.shape[0] * MILANO_CLEAN.shape[1]) * 100)
print(f"\n✅ COMPLETENESS: {completeness:.2f}%")

print(f"\n📈 NUMERICAL COLUMNS: {len(NUM)}")
for col in NUM:
    print(f"   - {col}: range [{MILANO_CLEAN[col].min():.2f}, {MILANO_CLEAN[col].max():.2f}]")

print(f"\n📝 CATEGORICAL COLUMNS: {len(CAT)}")
for col in CAT[:5]:  # Show first 5
    print(f"   - {col}: {MILANO_CLEAN[col].nunique()} unique values")
if len(CAT) > 5:
    print(f"   ... and {len(CAT) - 5} more columns")

# Check for duplicates
n_duplicates = MILANO_CLEAN.duplicated().sum()
print(f"\n🔄 DUPLICATES: {n_duplicates}")

print("\n" + "=" * 60)
print("Data cleaning pipeline completed successfully!")
print("=" * 60)